In [ ]:
!pip install -q rdkit
!pip install -q py3Dmol
!pip install -q pyscf
!pip install -q pythreejs
!pip install -q git+https://github.com/funkymunkycool/Cube-Toolz.git

In [ ]:
#@title Import Packages
import pathlib
import sys, os

# RDKit imports:
from rdkit import Chem
from rdkit.Chem import (
    AllChem,
    rdCoordGen,
)
from rdkit.Chem.Draw import IPythonConsole

IPythonConsole.ipython_useSVG = True  # Use higher quality images for molecules

# For visualization of molecules and orbitals:
import py3Dmol
import cube_tools
from pyscf import gto, scf, lo, tools
from google.colab import output
output.enable_custom_widget_manager()
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import ipywidgets as widgets
%matplotlib inline
sns.set_theme(style="ticks", context="talk", palette="muted")

# For numerics:
import numpy as np
import pandas as pd

In [ ]:
#@title PAH Selection
# load in the smiles of some PAHs
molecule_smiles = []
ethene = Chem.MolFromSmiles("cc"), 'Ethene'; molecule_smiles.append(ethene)
ring1 = Chem.MolFromSmiles("c1ccccc1"), 'Benzene'; molecule_smiles.append(ring1)
ring2 = Chem.MolFromSmiles("c1c2ccccc2ccc1"), 'Napthalene'; molecule_smiles.append(ring2)
ring3 = Chem.MolFromSmiles("c1ccc2cc3ccccc3cc2c1"), 'Anthracene'; molecule_smiles.append(ring3)
ring4 = Chem.MolFromSmiles("c1c2cc3cc4ccccc4cc3cc2ccc1"), 'Tetracene'; molecule_smiles.append(ring4)
ring5 = Chem.MolFromSmiles("c1ccc2cc3cc4cc5ccccc5cc4cc3cc2c1"), 'Pentacene'; molecule_smiles.append(ring5)
ring6 = Chem.MolFromSmiles("c1c2cc3cc4cc5cc6ccccc6cc5cc4cc3cc2ccc1"), 'Hexacene'; molecule_smiles.append(ring6)

def get_xyz(molecule, optimize=False):
    """Get xyz-coordinates for the molecule"""
    mol = Chem.Mol(molecule)
    mol = AllChem.AddHs(mol, addCoords=True)
    AllChem.EmbedMolecule(mol)
    if optimize:  # Optimize the molecules with the MM force field:
        AllChem.MMFFOptimizeMolecule(mol)
    xyz = []
    for lines in Chem.MolToXYZBlock(mol).split("\n")[2:]:
        strip = lines.strip()
        if strip:
            xyz.append(strip)
    xyz = "\n".join(xyz)
    return mol, xyz

print('1-benzene\n2-napthalene\n3-anthracene\n4-tetracene\n5-pentacene\n6-hexacene')
print('--------------------')
molecule_num = input('Select a linear PAH: \n')
molecule = molecule_smiles[int(molecule_num)][0]
molecule_name = molecule_smiles[int(molecule_num)][1]
molecule3d, xyz = get_xyz(molecule)
view = py3Dmol.view(
    data=Chem.MolToMolBlock(molecule3d),
    style={"stick": {}, "sphere": {"scale": 0.3}},
    width=400,
    height=300,
)

view.zoomTo()
view.show()



1-benzene
2-napthalene
3-anthracene
4-tetracene
5-pentacene
6-hexacene
--------------------
Select a linear PAH: 
2


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
#@title Hartree-Fock/STO-3G SCF Calculation
%time
def run_calculation(xyz, basis="3-21g"):
    """Calculate the energy (+ additional things like MO coefficients) with pyscf."""
    mol = gto.M(atom=xyz,basis=basis,unit="ANG",symmetry=True)
    mol.build()
    mf = scf.RHF(mol).run()
    mos = mf.mo_coeff
    #HL_gap_ev = HL_gap * 27.2114
    return mf, mol
mf, mol = run_calculation(xyz, basis="sto-3g")

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 12.2 µs
converged SCF energy = -378.668279846639


In [ ]:
#@title Frontier MO Energies
homo_num = np.count_nonzero(mf.mo_occ == 2)-1
lumo_num = np.count_nonzero(mf.mo_occ == 2)
homo_en = mf.mo_energy[homo_num]
mos_occ = mf.mo_coeff[:,:lumo_num]

lumo_en = mf.mo_energy[lumo_num]
mos_unocc = mf.mo_coeff[:,lumo_num:]
hl_gap = abs(lumo_en - homo_en)
hl_gap_ev = hl_gap * 27.2114

MO_labels = []

for i in range(len(mf.mo_occ)-lumo_num):
  if i == 0:
    MO_labels.append('LUMO')
  else:
    MO_labels.append(f'LUMO+{i}')

MO_labels.reverse()
  
for i in range(homo_num+1):
  if i == 0 :
    MO_labels.append('HOMO')
  else:
    MO_labels.append(f'HOMO-{i}')
MO_labels.reverse()


table = pd.DataFrame({"En (Ha)": mf.mo_energy, "Occ. ": mf.mo_occ.astype('int'), "Type": MO_labels})
table.index.name = 'MO #'

frontier_nums = np.arange(homo_num-4,lumo_num+5,1)
frontier_labels = table.iloc[frontier_nums]['Type'].tolist()

display(table.iloc[homo_num-4:lumo_num+5][::-1])
print('================================')
print(f'HOMO-LUMO Gap: {hl_gap:.3f} Ha (\033[1m\033[94m{hl_gap_ev:.3f} eV\033[0m)')

,En (Ha),Occ.,Type
MO #,,,
38,0.576580,0,LUMO+4
37,0.420797,0,LUMO+3
36,0.339214,0,LUMO+2
35,0.255779,0,LUMO+1
34,0.197985,0,LUMO
33,-0.219794,2,HOMO
32,-0.275131,2,HOMO-1
31,-0.338702,2,HOMO-2
30,-0.403189,2,HOMO-3


HOMO-LUMO Gap: 0.418 Ha (11.368 eV)


In [ ]:
#@title Generate Gaussian Cube Files (this is slow) 
good_isovalues = []

def get_orbital(i,label):
  tools.cubegen.orbital(mol, f'{label}.cube', mf.mo_coeff[:,i],nx=80,ny=80,nz=80)
  square_cube = cube_tools.cube(f'{label}.cube')
  pos = np.ma.array(square_cube.data, mask = square_cube.data < 0)
  good_isovalues.append([f'{label}',pos.mean()])
  square_cube.square_cube(power=2)
  pos = np.ma.array(square_cube.data, mask = square_cube.data < 0)
  good_isovalues.append([f'{label}_square',pos.mean()])
  square_cube.write_cube(f'{label}_square.cube')

for i in range(len(frontier_nums)):
  get_orbital(frontier_nums[i],frontier_labels[i])
good_isovalues = np.array(good_isovalues)

grid_data = {}
for name in frontier_labels:
  with open(f'{name}.cube') as f:
    grid_data[f'{name}'] = f.read()
  with open(f'{name}_square.cube') as f:
    grid_data[f'{name}_square'] = f.read()
  

2
2
2
2
2
2
2
2
2
2


In [ ]:
#@title Orbital Plotter (execute cell once)
orbital_select = widgets.Dropdown(value='HOMO',options=frontier_labels,description='Plot: ')

def draw_orbital(label,psi_square=False,show_noninteractive_png=False):
    
    view = py3Dmol.view(width=500,height=500)
    if psi_square:
      index = int(np.where(good_isovalues == f'{label}_square')[0])
      isoval = float(good_isovalues[index][1])
      print(f'\t isovalue: {isoval:.4f}')
      view.addVolumetricData(grid_data[f'{label}_square'], "cube", {'isoval': isoval, 'color': "red", 'opacity': 0.90})  
    else:
      with open(f'{label}.cube') as f:
        cube_data = f.read()
      index = int(np.where(good_isovalues == f'{label}')[0])
      isoval = float(good_isovalues[index][1])*2
      print(f'\t isovalue: {isoval:.4f}')
      view.addVolumetricData(grid_data[f'{label}'], "cube", {'isoval': isoval, 'color': "red", 'opacity': 0.90})
      view.addVolumetricData(grid_data[f'{label}'], "cube", {'isoval': -isoval, 'color': "blue", 'opacity': 0.90})
    view.addModel(Chem.MolToMolBlock(molecule3d), 'mol')
    view.setStyle({'stick':{}, 'sphere': {"scale":0.3}})

    view.show()
    if show_noninteractive_png:
      view.png()

print(f'\t {molecule_name} HF/STO-3G Orbitals')
print('\t===============================')

widgets.interactive(draw_orbital, {'manual': True, "manual_name": "Render"}, label=orbital_select)


	 Napthalene HF/STO-3G Orbitals


interactive(children=(Dropdown(description='Plot: ', index=4, options=('HOMO-4', 'HOMO-3', 'HOMO-2', 'HOMO-1',…